# Intraday Option Vol Surfaces

In [1]:
import os
import time
import pickle
import numpy as np
import pandas as pd
import concurrent.futures as cf
import matplotlib.pyplot as plt

from datetime import timedelta, datetime, date
from thetadata import ThetaClient, OptionReqType, OptionRight, DateRange, DataType

# Get all Expirations for SPY Options

In [2]:
your_username = 'mswaseem131@gmail.com'
your_password = 'Maisara@1234'

def get_expirations(root_ticker) -> pd.DataFrame:
    """Request expirations from a particular options root"""
    # Create a ThetaClient
    client = ThetaClient(username=your_username, passwd=your_password, timeout=15)

    # Connect to the Terminal
    with client.connect():

        # Make the request
        data = client.get_expirations(
            root=root_ticker,
        )

    return data

root_ticker = 'SPY'
expirations = get_expirations(root_ticker)

root_ticker = 'SPY'
expirations = get_expirations(root_ticker)

exp_dates = expirations[expirations > time_now + timedelta(days=7)]
exp_dates

If you require API support, feel free to join our discord server! http://discord.thetadata.us
Unable to fetch the latest terminal version. Please contact support.


ConnectionError: Unable to connect to the local Theta Terminal process. Try restarting your system.

# Get all Strikes for each SPY Option

In [3]:
def get_strikes(root_ticker, expiration_date) -> pd.DataFrame:
    """Request strikes from a particular option contract"""
    # Create a ThetaClient
    client = ThetaClient(username=your_username, passwd=your_password, timeout=15)

    # Connect to the Terminal
    with client.connect():

        # Make the request
        data = client.get_strikes(
            root=root_ticker,
            exp=expiration_date
        )

    return data

root_ticker = 'SPY'
exp_date = date(2022,9,16)
strikes = get_strikes(root_ticker, exp_date)
strikes

root_ticker = 'SPY'
all_strikes = {}
for exp_date in exp_dates:
    all_strikes[exp_date] = pd.to_numeric(get_strikes(root_ticker, exp_date))

If you require API support, feel free to join our discord server! http://discord.thetadata.us
[09-17-2023 14:13:08] INFO: Starting Theta Terminal v1.0.6 Revision Rev A...
[09-17-2023 14:13:08] INFO: Config dir: C:\Users\hp\ThetaData\ThetaTerminal
[09-17-2023 14:13:09] INFO: [MDDS] Attempting login as 
[09-17-2023 14:13:10] INFO: [MDDS] Disconnected from server: INVALID_CREDENTIALS
[09-17-2023 14:13:10] INFO: [FPSS] Attempting login as 
[09-17-2023 14:13:12] INFO: [FPSS] Disconnected from server: INVALID_CREDENTIALS


TimeoutError: timed out

# Find combined Strikes across expiry dates

In [4]:
combined_strikes = [list(strikes) for strikes in all_strikes.values()]
vol_surface_strikes = set.intersection(*map(set,combined_strikes))
vol_surface_strikes

with open('vol_surface_strikes.pkl', 'wb') as f:
    pickle.dump(vol_surface_strikes, f)

with open('vol_surface_strikes.pkl', 'rb') as f:
    vol_surface_strikes = pickle.load(f)

vol_surface_strikes = list(vol_surface_strikes)
vol_surface_strikes.sort()
vol_surface_strikes

NameError: name 'all_strikes' is not defined

# Retreive EOD Options Data

In [5]:
def end_of_day(root_ticker, exp_date, strike, from_date, to_date) -> pd.DataFrame:
    """Request end-of-day data"""
    # Create a ThetaClient
    client = ThetaClient(username=your_username, passwd=your_password, timeout=10)

    # Connect to the Terminal
    with client.connect():

        # Make the request
        data = client.get_hist_option(
            req=OptionReqType.EOD,
            root=root_ticker,
            exp=exp_date,
            strike=strike,
            right=OptionRight.CALL,
            date_range=DateRange(from_date, to_date),
        )

    return data

root_ticker = 'SPY'
from_date = date(2022,8,26)
to_date = date(2022,8,31)
exp_date = exp_dates.tolist()[0]
strike = 420000
data = end_of_day(root_ticker, exp_date, int(int(strike)/1000), from_date, to_date)
data

NameError: name 'exp_dates' is not defined

# Retreive Trades Data

In [6]:
# Make the request
def trades(root_ticker, exp_date, strike, from_date, to_date) -> pd.DataFrame:
    """Request trade level data"""
    # Create a ThetaClient
    client = ThetaClient(username=your_username, passwd=your_password, timeout=10)

    # Connect to the Terminal
    with client.connect():

        data = client.get_hist_option(
            req=OptionReqType.TRADE,
            root=root_ticker,
            exp=exp_date,
            strike=strike,
            right=OptionRight.CALL,
            date_range=DateRange(from_date, to_date),
            progress_bar=False,
        )

    return data


root_ticker = 'SPY'
exp_date = date(2022,9,16)
from_date = date(2022,8,31)
to_date = date(2022,8,31)
strike = 420000
data = trades(root_ticker, exp_date, int(int(strike)/1000), from_date, to_date)
data

If you require API support, feel free to join our discord server! http://discord.thetadata.us
Unable to fetch the latest terminal version. Please contact support.


ConnectionError: Unable to connect to the local Theta Terminal process. Try restarting your system.

# Use MS_OF_DAY to Create DateTime

In [7]:
data['DATETIME'] = data[DataType.DATE] + pd.TimedeltaIndex(data[DataType.MS_OF_DAY], unit='ms')
plt.figure(figsize=(12,6))
plt.title('Traded Price of 16-Sep SPY Call Options \n Trading day: 31-Aug ') 
plt.xlabel('Datetime')
plt.ylabel('Traded Price $')
plt.plot(data['DATETIME'],data[DataType.PRICE])

NameError: name 'data' is not defined

In [8]:
# Retreive Quoted Intraday Options Prices
# Make the request
def quotes(root_ticker, exp_date, strike, from_date, to_date, interval_size=0) -> pd.DataFrame:
    """Request quotes both bid/ask options data"""
    # Create a ThetaClient
    client = ThetaClient(username=your_username, passwd=your_password, timeout=10)
    # Connect to the Terminal
    with client.connect():
        data = client.get_hist_option(
            req=OptionReqType.QUOTE,
            root=root_ticker,
            exp=exp_date,
            strike=strike,
            right=OptionRight.CALL,
            date_range=DateRange(from_date, to_date),
            progress_bar=False,
            interval_size=interval_size
        )
    return data
root_ticker = 'SPY'
exp_date = date(2022,9,16)
from_date = date(2022,8,31)
to_date = date(2022,8,31)
strike = 420000
data = quotes(root_ticker, exp_date, int(int(strike)/1000), from_date, to_date)
data

If you require API support, feel free to join our discord server! http://discord.thetadata.us
[09-17-2023 14:17:28] INFO: Starting Theta Terminal v1.0.6 Revision Rev A...
[09-17-2023 14:17:28] INFO: Config dir: C:\Users\hp\ThetaData\ThetaTerminal
[09-17-2023 14:17:29] INFO: [MDDS] Attempting login as 
[09-17-2023 14:17:30] INFO: [MDDS] Disconnected from server: INVALID_CREDENTIALS
[09-17-2023 14:17:30] INFO: [FPSS] Attempting login as 
[09-17-2023 14:17:32] INFO: [FPSS] Disconnected from server: INVALID_CREDENTIALS


TimeoutError: timed out

In [9]:
# Change interval size


interval_size = 5*60000
data = quotes(root_ticker, exp_date, int(int(strike)/1000), from_date, to_date, interval_size)
dt_date_ms = data[DataType.DATE] + pd.TimedeltaIndex(data[DataType.MS_OF_DAY], unit='ms')
data.insert(loc=0, column='DATETIME', value=dt_date_ms)
data
morning_int = 120
afternoon_int = 168
morning = data['DATETIME'][morning_int]
afternoon = data['DATETIME'][afternoon_int]
print("Morning Interval", morning, "\nAfternoon Interval", afternoon)
data.iloc[afternoon_int]

If you require API support, feel free to join our discord server! http://discord.thetadata.us
Unable to fetch the latest terminal version. Please contact support.


ConnectionError: Unable to connect to the local Theta Terminal process. Try restarting your system.

In [ ]:
# Calculate Weighted Mid Price (Micro-Price)

In [ ]:
def weighted_mid_price(row):
    try:
        V_mid = row[DataType.ASK_SIZE] + row[DataType.BID_SIZE]
        x_a = row[DataType.ASK_SIZE]/V_mid
        x_b = 1 - x_a
        return row[DataType.ASK]*x_a + row[DataType.BID]*x_b
    except:
        return np.nan
midprices = data.apply(weighted_mid_price, axis=1)
midprices